In [1]:
!pip install insightface onnxruntime torch torchvision -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 13.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from numpy.linalg import norm
import os

In [3]:
app = FaceAnalysis(
    name="buffalo_l",     # ArcFace + detector + landmark
    providers=['CPUExecutionProvider']
)

app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 84190.24KB/s] 


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
REF_DIR = "/content/drive/MyDrive/dataset_face/me"
embeddings = []

for img_name in os.listdir(REF_DIR):
    img = cv2.imread(os.path.join(REF_DIR, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = app.get(img)
    if len(faces) == 0:
        continue

    embeddings.append(faces[0].embedding)

ref_embedding = np.mean(embeddings, axis=0)
ref_embedding = ref_embedding / norm(ref_embedding)

print("Referans embedding hazir:", ref_embedding.shape)

Referans embedding hazır: (512,)


In [5]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

In [6]:
def verify_face(image_path, threshold=0.6):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = app.get(img)
    if len(faces) == 0:
        return False, 0.0

    emb = faces[0].embedding
    emb = emb / norm(emb)

    score = cosine_similarity(ref_embedding, emb)
    return score > threshold, score

In [7]:
result, score = verify_face("/content/drive/MyDrive/test.jpg")

print("Eşleşti mi:", result)
print("Benzerlik:", score)

Eşleşti mi: True
Benzerlik: 0.6437473


In [8]:
import numpy as np

np.save("/content/drive/MyDrive/ref_embedding.npy", ref_embedding)
print("Referans embedding kaydedildi")

Referans embedding kaydedildi
